In [ ]:
#https://github.com/vense/keras-grad-cam/blob/master/grad-cam.py
#https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.4-visualizing-what-convnets-learn.ipynb

In [1]:
!nvidia-smi

Thu Feb 28 07:52:13 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.78       Driver Version: 410.78       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 22%   41C    P8    17W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:03:00.0  On |                  N/A |
| 45%   71C    P2    76W / 250W |  11761MiB / 12190MiB |     34%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import sys
sys.path.append("/home/Deep_Learner/work/network/keras_utils") 
%matplotlib notebook
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential, Model
from keras import optimizers
from keras import backend
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, AveragePooling2D, Concatenate
from image.image_data_generator import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from image.directory_iterator import DirectoryIterator
from image.dataframe_iterator import DataFrameIterator
from keras.applications.vgg16 import preprocess_input
from keras import applications
from keras import optimizers
from keras.metrics import categorical_accuracy
from keras import Model
from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K

import tensorflow as tf
import numpy as np
import keras
import sys
import cv2

Using TensorFlow backend.


it's a sequence


In [4]:
IMAGE_SIZE = 100
name_weights = '/home/Deep_Learner/work/local/VGG_dataset2_clr_ALL.h5'

model_name = 'VGG_GRAD'

base_path = ('/home/Deep_Learner/work/local/Models/{}'.format(model_name))
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [5]:
from sklearn.metrics import roc_curve, roc_auc_score, auc#plotting the receiver operating characteristics --> evaluate performance cutting point vice
def plot_roc(label, predictions, modelname, path): #IDEA: set diffrent cutting point based on ROC for ensembling   
    roc_auc_score(label, predictions)
    print('The ROC-Score is: {}' .format(roc_auc_score))

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(label, predictions)
    auc_keras = auc(fpr_keras, tpr_keras)
    #print(auc_keras)

    fig = plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve: {}' .format(auc_keras))
    plt.legend(loc='best')
    fig.savefig(path + '\ROC-Curve_{}.png' .format(modelname), dpi=300) #saving PLOT 
    plt.show()
    
from sklearn.metrics import confusion_matrix
import itertools
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [6]:
def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img = image.load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    x = image.img_to_array(img)
    x = (x - np.min(x))/np.ptp(x)
    x = np.expand_dims(x, axis=0)

    return x

#registering guidedrelu as a tensorflow gradient
#the modified gradient only backpropagates gradients which had a positive activation during forward pass
def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)


#Generating the Saliency Map of the Image with respect to the prediction & the given model        
def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    #Keras function returning the saliency map given an image input
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

#to implement guided backpropagation we need to duplicate the model with modified gradients & linear activation
#replacing the model activations with linear activations
#linear activation ensures that when computing derivatives of a class output, features which minimize the otehr classes arent encluded
def modify_backprop(model, name, name_weights):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = get_model_classif_VGG_base_trainable()
        new_model.load_weights(name_weights)
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
def get_model_classif_VGG_base_trainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    
    x = base_model_VGG.output
    x = GlobalMaxPooling2D()(x)
    x = (Dense(256, activation='relu'))(x)
    x = (Dropout(0.5)) (x)
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam()
    model_VGG.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=[categorical_accuracy])
    print('Model loaded.')
    return model_VGG

In [9]:
with tf.device('/gpu:0'):
    #model = VGG16(weights='imagenet')
    model = get_model_classif_VGG_base_trainable()
    model.load_weights(name_weights)

Model loaded.


In [10]:
from keras_preprocessing.image import ImageDataGenerator

batch_size= 1
datagen_test = ImageDataGenerator(rescale=1./255)
test_path = ('/home/Deep_Learner/work/local/Tiles_272/Validation/')
test_generator = datagen_test.flow_from_directory(test_path, 
                                                  batch_size = batch_size, 
                                                  shuffle=False,#FALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSE
                                                  target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                  class_mode='categorical')

with tf.device('/gpu:0'):
    model.load_weights('/home/Deep_Learner/work/local/VGG_dataset2_clr_ALL.h5')
    val_loss, val_acc = \
    model.evaluate_generator(test_generator, 
                                 steps=len(test_generator),
                                verbose=1,
                                workers = 5,
                                use_multiprocessing = False)

    print('val_loss:', val_loss)
    print('val_acc:', val_acc)

Found 4087 images belonging to 2 classes.
4087/4087 [==============================] - 45s 11ms/step
val_loss: 0.7686488606504186
val_acc: 0.8213848788842671


In [11]:
filenames = test_generator.filenames
print(filenames)

['a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD1.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD10.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD100.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD101.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD102.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD103.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD104.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD105.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD106.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD107.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD108.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD109.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD11.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD110.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD111.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD112.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD113.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD114.png', 'a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD115.pn

In [12]:
#preprocessed_input = load_image('/home/Deep_Learner/work/network/TSC1.png')
preprocessed_input = load_image('/home/Deep_Learner/work/local/Tiles_272/Validation/a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD1.png')

In [13]:
predictions = model.predict(preprocessed_input)
predicted_class = np.argmax(predictions)
print(predictions)
print(predicted_class)

[[0.00124188 0.9987581 ]]
1


In [48]:
def grad_cam(input_model, image, category_index, layer_name):
    #defining the layer & the position of our class on the prediction vector
    nb_classes = 2
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    x = input_model.layers[-1].output
    x = Lambda(target_layer, output_shape=target_category_loss_output_shape)(x)
    model = keras.models.Model(input_model.layers[0].input, x)
    
    #getting the output of the layers with the predefined layername
    loss = K.sum(model.layers[-1].output)
    conv_output = [l for l in model.layers if l.name is layer_name][0].output
    
    #Gradient for our class with regard to the output-feature map of the predefined layer
    grads = normalize(K.gradients(loss, conv_output)[0])
    #Keras function taking image input and accessing the quantities of:
    #"grads" (mean intensity gradient for a specific class) & 
    #"conv_output" (output featuremaps)
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])
    output, grads_val = gradient_function([image])
    #values of grads & conv_output as numpy arrays
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    #normalising gradients via GlobalPooling
    weights = np.mean(grads_val, axis = (0, 1))
    #initializing array with right shape
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)
    #multiplying each channel in the feature-map by "how important it is" with respect to the given class
    for i, w in enumerate(weights):
        cam += w * output[:, :, i]
    
    #formating for visualisation purposes: image, heatmap, GRAD CAM
    cam = cv2.resize(cam, (IMAGE_SIZE, IMAGE_SIZE))
    #sudo Relu
    cam = np.maximum(cam, 0)
    #Normalizing for Visualisation
    heatmap = cam / np.max(cam)
    heatmap = heatmap *255
    
    
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = 255 * image / np.max(image)

    cam = cv2.applyColorMap(np.uint8(heatmap), cv2.COLORMAP_JET)
    cam[np.where(cam < 95)] = 0
    #superimposed_img = cv2.addWeighted(image, 0.7, cam, 0.3, 0)
    cam = np.float32(cam) + np.float32(image)
    superimposed_img = 255 * cam / np.max(cam)
    #print(np.uint8(cam))
    return np.uint8(superimposed_img), heatmap

def guided_grad_cam(model, preprocessed_input, heatmap, name_weights):
    register_gradient()
    guided_model = modify_backprop(model, 'GuidedBackProp', name_weights=name_weights)
    saliency_fn = compile_saliency_function(guided_model)
    saliency = saliency_fn([preprocessed_input, 0])
    gradcam = saliency[0] * heatmap[..., np.newaxis]
    ggc= deprocess_image(gradcam)
    return ggc

In [49]:
%matplotlib notebook
cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")

plt.imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
#plt.imshow(np.uint8(heatmap*255), cmap='viridis')
#cv2.imwrite("gradcam.jpg", cam)
#cv2.imwrite("heatmap.jpg", heatmap)

<IPython.core.display.Javascript object>

In [50]:
%matplotlib notebook
ggc =  guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)   
plt.imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB), cmap= 'jet')
#plt.imshow(ggc)

Model loaded.


<IPython.core.display.Javascript object>

## Iterating over Test Data

In [51]:
batch_size= 1
datagen_test = ImageDataGenerator(rescale=1./255)
test_path = ('/home/Deep_Learner/work/local/Tiles_272/Validation/')
test_generator = datagen_test.flow_from_directory(test_path, 
                                                  batch_size = batch_size, 
                                                  shuffle=False,#FALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSEFALSE
                                                  target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                  class_mode='categorical')

Found 4087 images belonging to 2 classes.


In [52]:
import pandas as pd
from glob import glob 
from skimage.io import imread
from keras.utils import to_categorical
import gc
# Setup the data_frame: Path, ID, Label, Image
test_path = ('/home/Deep_Learner/work/local/Tiles_272/Validation/')
#test_path = base_path + '\\val_dir'
print(test_path)

fcd = test_path +  '/a_fcd'
tsc = test_path + '/b_tsc'
print(fcd)
#fcd folder
df = pd.DataFrame({'path': glob(os.path.join(fcd,'*.png'))})
df['id'] = df.path.map(lambda x: x.split('/')[9].split('.')[0]) 
df['label'] = 0

#tsc folder
df_pos = pd.DataFrame({'path': glob(os.path.join(tsc,'*.png'))})
df_pos['id'] = df_pos.path.map(lambda x: x.split('/')[9].split('.')[0]) 
df_pos['label'] = 1

#both
df_train = pd.concat([df, df_pos])

#add images
df_train['image'] = df_train['path'].map(imread)

print(len(df_train))
print(df_train.head())

/home/Deep_Learner/work/local/Tiles_272/Validation/
/home/Deep_Learner/work/local/Tiles_272/Validation//a_fcd
4087
                                                path  \
0  /home/Deep_Learner/work/local/Tiles_272/Valida...   
1  /home/Deep_Learner/work/local/Tiles_272/Valida...   
2  /home/Deep_Learner/work/local/Tiles_272/Valida...   
3  /home/Deep_Learner/work/local/Tiles_272/Valida...   
4  /home/Deep_Learner/work/local/Tiles_272/Valida...   

                                   id  label  \
0          N152 18 I1 - 2018-11-28 17      0   
1          N1173-17 3 - 2018-12-17 20      0   
2          N1173-17 3 - 2018-12-17 20      0   
3          N1173-17 3 - 2018-12-17 20      0   
4  N295-18 I4 (44-18) - 2018-12-18 13      0   

                                               image  
0  [[[255, 204, 226], [239, 181, 206], [218, 155,...  
1  [[[238, 171, 206], [240, 170, 206], [244, 179,...  
2  [[[230, 141, 187], [229, 149, 188], [226, 154,...  
3  [[[255, 187, 222], [255, 181, 223], 

In [53]:
#extracting images out of df
imgs = []
for img in df_train['image']:
    imgs.append(img)
        
X = np.asarray(imgs)
y = to_categorical(df_train['label'])
filenames = test_generator.filenames

print(X.shape)
print(y.shape)

(4087, 272, 272, 3)
(4087, 2)


In [54]:
test_generator.reset()

In [55]:
predictions = model.predict_generator(test_generator, 
                                          steps=len(test_generator), 
                                          workers=6, 
                                          use_multiprocessing=False, 
                                          verbose=1)

4087/4087 [==============================] - 54s 13ms/step


In [56]:
# Get the true labels
y_label = np.argmax(y, axis=1)
print(y_label)

df_preds = pd.DataFrame(predictions, columns=['FCD', 'TSC'])
print(df_preds.head())

# Get the predicted labels as probabilities
y_pred = df_preds['TSC']
y_predictions = np.asarray(y_pred.round(3))
print(y_predictions)

#rounding predictions + make them int --> compare to label / heatmap
y_pred = y_pred.round()
y_pred_int = []
for i in y_pred:
    if i == 1.0:
        y_pred_int.append(1)
    else:
        y_pred_int.append(0)

y_pred_int = np.asarray(y_pred_int)
print((y_pred_int.shape))

[0 0 0 ... 1 1 1]
        FCD           TSC
0  0.000348  9.996518e-01
1  0.999270  7.304159e-04
2  1.000000  1.642649e-10
3  0.996332  3.667998e-03
4  0.994321  5.679252e-03
[1.    0.001 0.    ... 0.001 0.002 0.   ]
(4087,)


In [61]:
%matplotlib notebook
import pycm
#maybe change pos/neg cutoff:
cm=ConfusionMatrix(actual_vector=y_label, predict_vector=y_predictions)
print(cm)

ModuleNotFoundError: No module named 'pycm'

In [59]:
!pip install pycm==1.9

In [ ]:
#maybe change pos/neg cutoff:
plot_roc(y_label, y_pred, modelname='try', path = base_path)

cm = confusion_matrix(y_label, y_pred.round())
cm_plot_labels = ['FCD', 'TSC']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix VGG')

In [23]:
filenames = test_generator.filenames
incorrect = np.where(y_pred!=y_label)[0]
for i, incorrect in enumerate(incorrect):
    print(incorrect)
    print(filenames[incorrect])

0
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD1.png
7
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD105.png
11
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD109.png
13
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD110.png
14
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD111.png
25
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD121.png
37
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD132.png
42
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD137.png
57
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD150.png
59
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD152.png
69
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD161.png
72
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD164.png
97
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD187.png
119
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD206.png
124
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD210.png
157
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD240.png
158
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD241.png
162
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FCD245.png
164
a_fcd/N1173-17 3 - 2018-12-17 20.11.35_FC

3102
b_tsc/N182-18 II1 (29 18) - 2018-12-11 09.21.14_TSC55.png
3103
b_tsc/N182-18 II1 (29 18) - 2018-12-11 09.21.14_TSC56.png
3108
b_tsc/N182-18 II1 (29 18) - 2018-12-11 09.21.14_TSC60.png
3111
b_tsc/N182-18 II1 (29 18) - 2018-12-11 09.21.14_TSC63.png
3113
b_tsc/N182-18 II1 (29 18) - 2018-12-11 09.21.14_TSC8.png
3146
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC127.png
3223
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC197.png
3264
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC233.png
3301
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC267.png
3331
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC294.png
3380
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC338.png
3385
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC342.png
3417
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC371.png
3461
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC410.png
3467
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC416.png
3508
b_tsc/N316 18 3 (47 18) - 2018-12-04 12.06.45_TSC453.png
3516

In [24]:
def load_i(path):
    img = image.load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img = (img - np.min(img))/np.ptp(img)
    x = image.img_to_array(img)
    x = (x - np.min(x))/np.ptp(x)
    x = np.expand_dims(x, axis=0)
    return x, img


In [ ]:
%matplotlib notebook
z = 1
preprocessed_input, original = load_i(test_path + filenames[z])
predicted_class = y_pred_int[z]
cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
plt.imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
#plt.imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB))

In [26]:
model_name = 'VGG_GRADCAM1'

GRAD_CAM_PATH = ('/home/Deep_Learner/work/local/{}'.format(model_name))
if not os.path.exists(GRAD_CAM_PATH):
    os.makedirs(GRAD_CAM_PATH)

In [ ]:
mostconfident = np.where((y_label==0) & (y_label==y_pred_int))[0]
print(len(mostconfident))

%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np

fig, ax = plt.subplots(4, 4, figsize=(20,10), squeeze=False)
fig.subplots_adjust(hspace=0.7, wspace=0.1)
plt.suptitle('GRAD CAM', fontsize=16)

for i , correct in enumerate(mostconfident[:1]):
    preprocessed_input, original = load_i(test_path + filenames[correct])
    predicted_class = y_pred_int[correct]
    cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
    ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
    ax[0,i].set_title(filenames[correct], fontsize=10)
    ax[0,i].imshow(original)
    
    ax[1,i].set_title("Predicted {}, Class {}".format(y_pred_int[correct], y_label[correct]), fontsize=10)
    map = ax[1,i].imshow(heatmap, interpolation="none", cmap = 'rainbow')
    
    ax[2,i].imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
    ax[3,i].imshow(ggc)

    ax[0,0].set_ylabel('Original')
    ax[1,0].set_ylabel('Heatmap')
    ax[2,0].set_ylabel('GRAD CAM')
    ax[3,0].set_ylabel('Guided GRAD CAM')
fig.savefig(GRAD_CAM_PATH + '/Mostconfident_Images_{}.png' .format(correct), dpi=100)

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np

fig, ax = plt.subplots(4, 4, figsize=(20,10), squeeze=False)
fig.subplots_adjust(hspace=0.7, wspace=0.1)
plt.suptitle('GRAD CAM', fontsize=16)

k=0
j=4

for n in range(len(mostconfident)):
    for i , correct in enumerate(mostconfident[k:j]):
        print(i)
        k+=4
        j+=4
        preprocessed_input, original = load_i(test_path + filenames[correct])
        predicted_class = y_pred_int[correct]
        cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
        ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
        ax[0,i].set_title(filenames[correct], fontsize=8)
        ax[0,i].imshow(original)

        ax[1,i].set_title("Predicted: {}, Label: {}".format(y_predictions[correct], y_label[correct]), fontsize=8)
        map = ax[1,i].imshow(heatmap, interpolation="none", cmap = 'rainbow')

        ax[2,i].imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
        ax[3,i].imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB))

        ax[0,0].set_ylabel('Original')
        ax[1,0].set_ylabel('Heatmap')
        ax[2,0].set_ylabel('GRAD CAM')
        ax[3,0].set_ylabel('Guided GRAD CAM')
    fig.savefig(GRAD_CAM_PATH + '/Mostconfident_Images_{}-{}.png' .format(k,j), dpi=100)

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np
correct_tsc = np.where((y_label==1) & (y_label==y_pred_int))[0]
print("Found {} confident correct TSC labels: {}" .format(len(correct_tsc), correct_tsc))

fig, ax = plt.subplots(4, 4, figsize=(20,10), squeeze=False)
fig.subplots_adjust(hspace=0.7, wspace=0.1)
plt.suptitle('GRAD CAM', fontsize=16)

k=0
j=4

for n in range(len(correct_tsc)):
    for i , correct in enumerate(correct_tsc[k:j]):
        print('Getting GRAD-CAMs for TSC {}- {}'.format(k,j))
        k+=4
        j+=4
        preprocessed_input, original = load_i(test_path + filenames[correct])
        predicted_class = y_pred_int[correct]
        cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
        ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
        ax[0,i].set_title(filenames[correct], fontsize=8)
        ax[0,i].imshow(original)

        ax[1,i].set_title("Predicted: {}, Label: {}".format(y_predictions[correct], y_label[correct]), fontsize=8)
        map = ax[1,i].imshow(heatmap, interpolation="none", cmap = 'rainbow')

        ax[2,i].imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
        ax[3,i].imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB))

        ax[0,0].set_ylabel('Original')
        ax[1,0].set_ylabel('Heatmap')
        ax[2,0].set_ylabel('GRAD CAM')
        ax[3,0].set_ylabel('Guided GRAD CAM')
    fig.savefig(GRAD_CAM_PATH + '/TSC_Mostconfident_Images_{}-{}.png' .format(k,j), dpi=100)

Found 1525 confident correct TSC labels: [2065 2066 2067 ... 4064 4076 4077]


<IPython.core.display.Javascript object>

Getting GRAD-CAMs for TSC 0- 4
Model loaded.
Getting GRAD-CAMs for TSC 4- 8
Model loaded.
Getting GRAD-CAMs for TSC 8- 12
Model loaded.
Getting GRAD-CAMs for TSC 12- 16
Model loaded.
Getting GRAD-CAMs for TSC 16- 20
Model loaded.
Getting GRAD-CAMs for TSC 20- 24
Model loaded.
Getting GRAD-CAMs for TSC 24- 28
Model loaded.
Getting GRAD-CAMs for TSC 28- 32
Model loaded.
Getting GRAD-CAMs for TSC 32- 36
Model loaded.
Getting GRAD-CAMs for TSC 36- 40
Model loaded.
Getting GRAD-CAMs for TSC 40- 44
Model loaded.
Getting GRAD-CAMs for TSC 44- 48
Model loaded.
Getting GRAD-CAMs for TSC 48- 52
Model loaded.
Getting GRAD-CAMs for TSC 52- 56
Model loaded.
Getting GRAD-CAMs for TSC 56- 60
Model loaded.
Getting GRAD-CAMs for TSC 60- 64
Model loaded.
Getting GRAD-CAMs for TSC 64- 68
Model loaded.
Getting GRAD-CAMs for TSC 68- 72
Model loaded.
Getting GRAD-CAMs for TSC 72- 76
Model loaded.
Getting GRAD-CAMs for TSC 76- 80
Model loaded.
Getting GRAD-CAMs for TSC 80- 84
Model loaded.
Getting GRAD-CAMs 